## Game of Nim
In this Python Notebook we will explore different ways of representing the *Game of Nim** in Python with hopes of extrapolating an optimal strategy to playing the game.

In [160]:
# import required libraries
import random as r
from collections import Counter

## Random Two Player Game 

In [161]:
# Setup the amount in each pile
piles = [10,14,18]

In [162]:
# random approach

while True:

  selectable1 = False
  selectable2 = False


  # Player 1's Turn

  # Check to see if there is one remaing pile at the end of Player 1's Turn
  if piles.count(0) == 2:
    player1 = piles.index(max(piles)) + 1
    amount1 = max(piles)

  else:
    while selectable1 == False:
      player1 = r.randint(1,3)

      if piles[player1-1] != 0:
        selectable1 = True
    amount1 = r.randint(1,piles[player1 - 1])

  piles[player1 - 1] = piles[player1 - 1] - amount1
  
  # Display Player 1's actions and resulting game state
  print("Player 1: takes " + str(amount1) + " from pile " + str(player1))
  print("The current state is: " + str(piles))
  
  # Check if Player 1 wins by end of their turn
  if sum(piles) == 0:
    print("Player 1 Wins!")
    break

  # Player 2's Turn

  # Check to see if there is one remaing pile at the beginning of Player 2's turn
  if piles.count(0) == 2:
    player2 = piles.index(max(piles)) + 1
    amount2 = max(piles)
  else:
    while selectable2 == False:
      player2 = r.randint(1,3)

      if piles[player2-1] != 0:
        selectable2 = True
    amount2 = r.randint(1,piles[player2 - 1])
  piles[player2 - 1] = piles[player2 - 1] - amount2

  # Display Player 2's actions and resulting game state
  print("Player 2: takes " + str(amount2) + " from pile " + str(player2))
  print("The current state is: " + str(piles))

  # Check if Player 2 wins by end of their turn
  if sum(piles) == 0:
    print("Player 2 Wins!")
    break

Player 1: takes 10 from pile 2
The current state is: [10, 4, 18]
Player 2: takes 8 from pile 1
The current state is: [2, 4, 18]
Player 1: takes 15 from pile 3
The current state is: [2, 4, 3]
Player 2: takes 2 from pile 2
The current state is: [2, 2, 3]
Player 1: takes 1 from pile 2
The current state is: [2, 1, 3]
Player 2: takes 1 from pile 3
The current state is: [2, 1, 2]
Player 1: takes 2 from pile 1
The current state is: [0, 1, 2]
Player 2: takes 1 from pile 3
The current state is: [0, 1, 1]
Player 1: takes 1 from pile 2
The current state is: [0, 0, 1]
Player 2: takes 1 from pile 3
The current state is: [0, 0, 0]
Player 2 Wins!


## AI Approach

In [163]:
# AI approach

# Board State
piles = [3, 3, 3]

In [164]:
# Entumerate all possible actions for player
def actions(s):
    return [(i,j) for j in range(len(piles)) if piles[j] != 0 for i in range(1,piles[j]+1)]

In [165]:
# Update Board State
def update(s,a):
    (amount, pile) = a
    s_copy = s.copy()
    s_copy[pile] = s_copy[pile] - amount
    return s_copy

In [166]:
# Checks if game is over
def terminal(s):
    if sum(s) == 0:
        return True
    else:
        return False

In [167]:
def utility(s, cost):
    term = terminal(s)
    if term is True:
        # Return the cost of reaching the terminal state
        return (term, cost)
        
    action_list = actions(s)
    utils = []
    for action in action_list:
        print("Run")
        new_s = update(s, action)
        # Every recursion will be an increment in the cost (depth)
        utils.append(utility(new_s, cost + 1))
        
    # Remember the associated cost with the score of the state.
    score = utils[0][0]
    idx_cost = utils[0][1]
    
    for i in range(len(utils)):
        if utils[i][0] > score:
            score = utils[i][0]
            idx_cost = utils[i][1]

     # Return the score with the associated cost.
    return (score, idx_cost) 

In [168]:
def minimax(s):
  action_list = actions(s)
  utils = []
  for action in action_list:
    new_s = update(s, action)
    utils.append((action, utility(new_s, 1)))
  # Each item in utils contains the action associated
  # the score and "cost" of that action.
  
  # if utils has no objects, then return a default action and utility
  if len(utils) == 0:
    return ((0, 0), (0, 0))

  # Sort the list in ascending order of cost.
  sorted_list = sorted(utils, key=lambda l : l[0][1])
  # Since the computer shall play 2nd
  # It is safe to return the object with minimum score.
  action = min(sorted_list, key = lambda l : l[1])
  return action

In [169]:
s = piles
while terminal(s) == False:

    print(s)
    # Human Player Goes First
    pile = int(input("Enter the Pile you wish to select from: "))
    amount = int(input("Enter the amount you wish to take: "))

    s = update(s, (amount, pile-1))

    # AI Goes Second
    print('\n\nThe is computer is playing its turn')
    # Get the action by running the minimax algorithm
    print(s)
    action = minimax(s)
    # Apply the returned action to the state and print the board
    s = update(s, action[0])

    

[3, 3, 3]


The is computer is playing its turn
[1, 3, 3]
Run
[-1, 3, 3]
Run
[-2, 3, 3]
Run
[-3, 3, 3]
Run
[-4, 3, 3]
Run
[-5, 3, 3]
Run
[-6, 3, 3]
Run
[-7, 3, 3]
Run
[-8, 3, 3]
Run
[-9, 3, 3]
Run
[-10, 3, 3]
Run
[-11, 3, 3]
Run
[-12, 3, 3]
Run
[-13, 3, 3]
Run
[-14, 3, 3]
Run
[-15, 3, 3]
Run
[-16, 3, 3]
Run
[-17, 3, 3]
Run
[-18, 3, 3]
Run
[-19, 3, 3]
Run
[-20, 3, 3]
Run
[-21, 3, 3]
Run
[-22, 3, 3]
Run
[-23, 3, 3]
Run
[-24, 3, 3]
Run
[-25, 3, 3]
Run
[-26, 3, 3]
Run
[-27, 3, 3]
Run
[-28, 3, 3]
Run
[-29, 3, 3]
Run
[-30, 3, 3]
Run
[-31, 3, 3]
Run
[-32, 3, 3]
Run
[-33, 3, 3]
Run
[-34, 3, 3]
Run
[-35, 3, 3]
Run
[-36, 3, 3]
Run
[-37, 3, 3]
Run
[-38, 3, 3]
Run
[-39, 3, 3]
Run
[-40, 3, 3]
Run
[-41, 3, 3]
Run
[-42, 3, 3]
Run
[-43, 3, 3]
Run
[-44, 3, 3]
Run
[-45, 3, 3]
Run
[-46, 3, 3]
Run
[-47, 3, 3]
Run
[-48, 3, 3]
Run
[-49, 3, 3]
Run
[-50, 3, 3]
Run
[-51, 3, 3]
Run
[-52, 3, 3]
Run
[-53, 3, 3]
Run
[-54, 3, 3]
Run
[-55, 3, 3]
Run
[-56, 3, 3]
Run
[-57, 3, 3]
Run
[-58, 3, 3]
Run
[-59, 3, 3]
Run
[-6

RecursionError: maximum recursion depth exceeded while calling a Python object

In [ ]:
piles=[0,0,0]
actions(piles)

[]